## Day 17

https://adventofcode.com/2024/day/17

In [16]:
import re

def read_input_17(filename):
    contents = """Register A: 30553366
Register B: 0
Register C: 0

Program: 2,4,1,1,7,5,4,7,1,4,0,3,5,5,3,0
"""
    inp = contents.split("\n\n")
    regs = [ int(re.findall(r"\d+",i)[0]) for i in inp[0].split("\n") ]
    prog = [ int(i) for i in re.findall(r"\d+",inp[1]) ]
    return regs, prog

In [17]:
def opcode(reg, prog, verbose=False, part=1):
    i = 0
    output = []

    while True:    
        op = prog[i]
        li = prog[i+1]
        
        # combo from literal
        co = li 
        if co>3 and co<7:
            co=reg[co-4]        
        
        if verbose:
            print(f"{i} | {op} : {co} | ",end="")
        
        if op==0: # adv
            reg[0] = reg[0] // 2**co  
        elif op==1: # bxl
            reg[1] = reg[1] ^ li 
        elif op==2: # bst
            reg[1] = co % 8
        elif op==3: # jnz
            if reg[0]!=0: 
                i = li - 2
        elif op==4: # bxc
            reg[1] = reg[1] ^ reg[2]
        elif op==5: # out
            output += [ co % 8 ]
            #if part==2 and output != prog[:len(output)]:
            #    return None, None
        elif op==6: # bdv
            reg[1] = reg[0] // 2**co  
        elif op==7: # cdv
            reg[2] = reg[0] // 2**co  

        if verbose:
            print(reg)

        i+=2
        if i>=len(prog):
            break
    return output,reg

In [18]:
def part1(filename):
    reg, prog = read_input_17(filename)
    output, _ = opcode(reg, prog)
    return "".join([ str(o)+',' for o in output ])[:-1]

In [19]:
print("Part 1:",part1("AOC2024inputs/input17.txt"))

Part 1: 1,3,7,4,6,4,2,3,5


### Part 2

In [14]:
def part2_bruteforce(filename):
    reg, prog = read_input_17(filename)
    i = 1
    while True:
        reg[0] = i
        output, reg = opcode(reg, prog)
        if output==prog:
            return i      
        i+=1

In [15]:
print("Test 2:",part2_bruteforce("examples/example17-2.txt"))

KeyboardInterrupt: 

### Iterative / recursive solution

The program works in base 8, and I notice that I need a input A value of the between $8^n-1$ and $8^{(n-1)}$ to produce n digits as output. Each digit is produced (almost) independently, and the least significant 3 bits of the input A value controls the first output digit. I can try to recursively build the initial value of A by searching which value between 0 and 7 (least significant 3 bits) generates the needed value, then accumulate it for the next output value by increasing by a factor 8 (as the value was actually represented in base 8).

In [20]:
filename = "AOC2024inputs/input17.txt"
reg_init, prog = read_input_17(filename)

#reg[0] = 8**len(prog)-1
reg[0] = 8**(len(prog)-1)

output, reg = opcode(reg,prog)
print(output)
print(prog)

NameError: name 'reg' is not defined

In [1031]:
#a = 0
#a = 5
a = 46

b = 0
c = 0

for i in range(8):
    aa = a * 8 + i
    reg_i = [ aa, b, c ]
    output, reg = opcode(reg_i,prog)
    print(aa,output)

368 [5, 3, 0]
369 [5, 3, 0]
370 [1, 3, 0]
371 [2, 3, 0]
372 [2, 3, 0]
373 [7, 3, 0]
374 [1, 3, 0]
375 [7, 3, 0]


In [21]:
def findA(prog,a=0,b=0,c=0,ip=-1):
    if abs(ip) > len(prog): 
      return a
    for i in range(8):
        aa = a * 8 + i
        reg = [ aa , b, c ]
        output, reg = opcode(reg,prog)
        if output[0]==prog[ip]:
            aa = findA(prog, aa, reg[1], reg[2], ip-1)
            if aa:
                return aa
    return None

def part2(filename):
    _, prog = read_input_17(filename)
    return findA(prog)

In [22]:
print("Part 2:",part2("AOC2024inputs/input17.txt"))

Part 2: 202367025818154


### Program reverse engineering

An alternative idea would be to reverse engineer the program, and to build a function acting like the inverse of its beaviour, then use it to search for the smallest solution (since there would be many, since some steps are not exclusive, e.g. % or // operations)

In [974]:
opnames = {
    0: 'adv  | A = A // 2**COM',
    1: 'bxl  | B = B ^ LIT',
    2: 'bst  | B = COM % 8',
    3: 'jnz  | if A!=0: goto LIT',
    4: 'bxc  | B = B^C',
    5: 'out  | print( COM % 8 )',
    6: 'bdv  | B = A//2**COM',
    7: 'cdv  | C = A//2**COM'
}

filename = "AOC2024inputs/input17.txt"
reg_init, prog = read_input_17(filename)

print(f"   | OP LIT COM | name | operation")
print("-----------------------------------------------")
for i in range(0,len(prog),2):
    op = prog[i]
    li = prog[i+1]
    co = li 
    if co==4:
        co = 'A'
    if co==5:
        co = 'B'
    if co==6:
        co = 'C' 
    print(f"{i:2d} | {op:2d}  {li:2d}   {co} | {opnames[op]}")

   | OP LIT COM | name | operation
-----------------------------------------------
 0 |  2   4   A | bst  | B = COM % 8
 2 |  1   1   1 | bxl  | B = B ^ LIT
 4 |  7   5   B | cdv  | C = A//2**COM
 6 |  4   4   A | bxc  | B = B^C
 8 |  1   4   A | bxl  | B = B ^ LIT
10 |  0   3   3 | adv  | A = A // 2**COM
12 |  5   5   B | out  | print( COM % 8 )
14 |  3   0   0 | jnz  | if A!=0: goto LIT


In [975]:
# 7,4,2,0,5,0,5,3,7

def program(A=46337277, B=0, C=0):
    output = []
    while True:
        B = A%8 
        B = B^1  
        C = A//2**B
        B = B^C
        B = B^4
        A = A//8 
        output.append(B%8)
        if A==0:
            break
    return output

program()

[7, 4, 2, 0, 5, 0, 5, 3, 7]